En este archivo puedes escribir lo que estimes conveniente. Te recomendamos detallar tu solución y todas las suposiciones que estás considerando. Aquí puedes ejecutar las funciones que definiste en los otros archivos de la carpeta src, medir el tiempo, memoria, etc.

In [1]:
file_path = "farmers-protest-tweets-2021-2-4.json"

In [2]:
from q1_memory import q1_memory

In [3]:
print(q1_memory(file_path))

[(datetime.date(2021, 2, 12), 'RanbirS00614606'), (datetime.date(2021, 2, 13), 'MaanDee08215437'), (datetime.date(2021, 2, 17), 'RaaJVinderkaur'), (datetime.date(2021, 2, 16), 'jot__b'), (datetime.date(2021, 2, 14), 'rebelpacifist'), (datetime.date(2021, 2, 18), 'neetuanjle_nitu'), (datetime.date(2021, 2, 15), 'jot__b'), (datetime.date(2021, 2, 20), 'MangalJ23056160'), (datetime.date(2021, 2, 23), 'Surrypuria'), (datetime.date(2021, 2, 19), 'Preetm91')]


In [4]:
%load_ext memory_profiler

The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler


In [5]:
%mprun -f q1_memory q1_memory(file_path)

Filename: /app/src/q1_memory.py

Line #    Mem usage    Increment  Occurrences   Line Contents
     6    109.4 MiB    109.4 MiB           1   def q1_memory(file_path: str) -> List[Tuple[datetime.date, str]]:
     7    109.4 MiB      0.0 MiB           1       records = []
     8   1156.7 MiB      0.0 MiB           2       with open(f"/app/data/{file_path}") as jfile:
     9   1156.7 MiB     96.1 MiB      117408           for line in jfile:
    10   1156.7 MiB    951.1 MiB      117407               records.append(json.loads(line))
    11   1186.5 MiB     29.9 MiB      117410       clean_records = [{"date": x["date"][:10], "username": x["user"]["username"]} for x in records]
    12   1189.2 MiB      2.6 MiB      234815       ordered_records = sorted(clean_records, key=lambda x: x["date"])
    13   1189.2 MiB      0.0 MiB      234830       count_by_date = [(k, len(list(g))) for k, g in (groupby(ordered_records, lambda x: x["date"]))]
    14   1189.2 MiB      0.0 MiB          27       top_c

In [7]:
%time q1_memory(file_path)

CPU times: user 5.06 s, sys: 859 ms, total: 5.92 s
Wall time: 5.92 s


[(datetime.date(2021, 2, 12), 'boycott_twiter'),
 (datetime.date(2021, 2, 13), 'Preetm91'),
 (datetime.date(2021, 2, 17), 'Monica_Gill1'),
 (datetime.date(2021, 2, 16), 'Monica_Gill1'),
 (datetime.date(2021, 2, 14), 'GurmVicky'),
 (datetime.date(2021, 2, 18), 'c9DQOwxxVsxUnIX'),
 (datetime.date(2021, 2, 15), 'bishnoimukesh_'),
 (datetime.date(2021, 2, 20), 'kpreet11'),
 (datetime.date(2021, 2, 23), 'Preetm91'),
 (datetime.date(2021, 2, 19), 'Preetm91')]

In [8]:
import cProfile
cProfile.run('q1_memory(file_path)')

         3093602 function calls in 6.107 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.000    0.000 <frozen codecs>:260(__init__)
        1    0.000    0.000    0.000    0.000 <frozen codecs>:309(__init__)
    49772    0.042    0.000    0.096    0.000 <frozen codecs>:319(decode)
        1    0.218    0.218    6.106    6.106 <string>:1(<module>)
   117407    0.106    0.000    4.419    0.000 __init__.py:299(loads)
       10    0.000    0.000    0.000    0.000 _strptime.py:26(_getlang)
       10    0.000    0.000    0.001    0.000 _strptime.py:309(_strptime)
       10    0.000    0.000    0.001    0.000 _strptime.py:565(_strptime_datetime)
   117407    0.175    0.000    4.275    0.000 decoder.py:332(decode)
   117407    3.976    0.000    3.976    0.000 decoder.py:343(raw_decode)
       10    0.000    0.000    0.000    0.000 locale.py:396(normalize)
       10    0.000    0.000    0.000    0.00

In [1]:
from q1_memory import q1_memory

In [12]:
%mprun -f q1_memory q1_memory(file_path)

Filename: /app/src/q1_memory.py

Line #    Mem usage    Increment  Occurrences   Line Contents
     5    109.6 MiB    109.6 MiB           1   def q1_memory(file_path: str) -> List[Tuple[datetime.date, str]]:
     6    109.6 MiB      0.0 MiB           1       records = []
     7   1156.5 MiB      0.0 MiB           2       with open(f"/app/data/{file_path}") as jfile:
     8   1156.5 MiB    100.2 MiB      117408           for line in jfile:
     9   1156.5 MiB    946.6 MiB      117407               records.append(json.loads(line))
    10   1156.5 MiB      0.0 MiB           1       return 1

In [6]:
%mprun -f q1_memory q1_memory(file_path)

Filename: /app/src/q1_memory.py

Line #    Mem usage    Increment  Occurrences   Line Contents
     5    100.4 MiB    100.4 MiB           1   def q1_memory(file_path: str) -> List[Tuple[datetime.date, str]]:
     6    100.4 MiB      0.0 MiB           1       records = []
     7    505.3 MiB      0.0 MiB           2       with open(f"/app/data/{file_path}") as jfile:
     8    505.3 MiB    399.5 MiB      117408           for line in jfile:
     9    505.3 MiB      5.4 MiB      117407               records.append(line)
    10                                                     #records.append(json.loads(line))
    11    505.3 MiB      0.0 MiB           1       return 1

In [13]:
import cProfile
cProfile.run('q1_memory(file_path)')

         1391029 function calls in 5.507 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.000    0.000 <frozen codecs>:260(__init__)
        1    0.000    0.000    0.000    0.000 <frozen codecs>:309(__init__)
    49772    0.040    0.000    0.094    0.000 <frozen codecs>:319(decode)
        1    0.291    0.291    5.507    5.507 <string>:1(<module>)
   117407    0.105    0.000    4.426    0.000 __init__.py:299(loads)
   117407    0.177    0.000    4.282    0.000 decoder.py:332(decode)
   117407    3.979    0.000    3.979    0.000 decoder.py:343(raw_decode)
        1    0.656    0.656    5.216    5.216 q1_memory.py:5(q1_memory)
    49772    0.054    0.000    0.054    0.000 {built-in method _codecs.utf_8_decode}
        1    0.000    0.000    5.507    5.507 {built-in method builtins.exec}
   117407    0.012    0.000    0.012    0.000 {built-in method builtins.isinstance}
   117407    0.015    0.0

In [7]:
import cProfile
cProfile.run('q1_memory(file_path)')

         216959 function calls in 0.892 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.000    0.000 <frozen codecs>:260(__init__)
        1    0.000    0.000    0.000    0.000 <frozen codecs>:309(__init__)
    49772    0.030    0.000    0.100    0.000 <frozen codecs>:319(decode)
        1    0.014    0.014    0.892    0.892 <string>:1(<module>)
        1    0.728    0.728    0.878    0.878 q1_memory.py:5(q1_memory)
    49772    0.070    0.000    0.070    0.000 {built-in method _codecs.utf_8_decode}
        1    0.000    0.000    0.892    0.892 {built-in method builtins.exec}
        1    0.037    0.037    0.037    0.037 {built-in method io.open}
        1    0.000    0.000    0.000    0.000 {method '__exit__' of '_io._IOBase' objects}
   117407    0.013    0.000    0.013    0.000 {method 'append' of 'list' objects}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.

In [3]:
records = q1_memory(file_path)

In [7]:
clean_records = [{"date": x["date"][:10], "username": x["user"]["username"]} for x in records]

In [8]:
clean_records

[{'date': '2021-02-24', 'username': 'ArjunSinghPanam'},
 {'date': '2021-02-24', 'username': 'PrdeepNain'},
 {'date': '2021-02-24', 'username': 'parmarmaninder'},
 {'date': '2021-02-24', 'username': 'anmoldhaliwal'},
 {'date': '2021-02-24', 'username': 'KotiaPreet'}]

In [9]:
def key_func(x):
    return x["date"]

In [10]:
from itertools import groupby

In [17]:
for k, g in (groupby(clean_records, lambda x: x["date"])):
    print(k)
    print(len(list(g)))

2021-02-24
5


In [2]:
from q1_memory import q1_memory
records = q1_memory(file_path)

In [7]:
top_count_by_day = sorted(records, key=lambda x: x[1], reverse=True)[:10]

In [4]:
records

[('2021-02-12', 12347),
 ('2021-02-13', 11296),
 ('2021-02-14', 10249),
 ('2021-02-15', 9197),
 ('2021-02-16', 10443),
 ('2021-02-17', 11087),
 ('2021-02-18', 9625),
 ('2021-02-19', 8204),
 ('2021-02-20', 8502),
 ('2021-02-21', 7532),
 ('2021-02-22', 7071),
 ('2021-02-23', 8417),
 ('2021-02-24', 3437)]

In [13]:
import json
from itertools import groupby

file_path = "farmers-protest-tweets-2021-2-4.json"

def q1_test(file_path):
    records = []
    with open(f"/app/data/{file_path}") as jfile:
        for line in jfile:
            records.append(json.loads(line))
    clean_records = [{"date": x["date"][:10], "username": x["user"]["username"]} for x in records]
    ordered_records = sorted(clean_records, key=lambda x: x["date"])
    count_by_date = [(k, len(list(g))) for k, g in (groupby(ordered_records, lambda x: x["date"]))]
    top_count_by_day = sorted(count_by_date, key=lambda x: x[1], reverse=True)[:10]

    count_by_date_and_user = [(k, len(list(g))) for k, g in (groupby(ordered_records, lambda x: (x["date"],x["username"])))]

    results = []
    for top_date in top_count_by_day:
        filter_group = filter(lambda x: x[0][0] == top_date[0], count_by_date_and_user)
        username = max(filter_group, key=lambda x: x[1])[0][1]
        results.append((top_date[0], username))

    return results

results = q1_test(file_path)
        

In [14]:
results

[('2021-02-12', 'boycott_twiter'),
 ('2021-02-13', 'Preetm91'),
 ('2021-02-17', 'Monica_Gill1'),
 ('2021-02-16', 'Monica_Gill1'),
 ('2021-02-14', 'GurmVicky'),
 ('2021-02-18', 'c9DQOwxxVsxUnIX'),
 ('2021-02-15', 'bishnoimukesh_'),
 ('2021-02-20', 'kpreet11'),
 ('2021-02-23', 'Preetm91'),
 ('2021-02-19', 'Preetm91')]

In [16]:
from datetime import datetime
datetime.strptime('2014-12-04', '%Y-%m-%d').date()

datetime.date(2014, 12, 4)